In [4]:
from functools import partial
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, regexp_replace, flatten, explode, struct, create_map, array
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, TimestampType


In [5]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType, StringType

In [6]:
spark = SparkSession.builder.appName('chap_2').master("local[*]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/01 18:20:11 WARN Utils: Your hostname, Sai-Sundar-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.78 instead (on interface en0)
25/07/01 18:20:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/01 18:20:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from faker import Faker
import pandas as pd
fake = Faker()
def generate_data(num):
    row = [{"name":fake.name(),
           "address":fake.address(),
           "city":fake.city(),
           "state":fake.state(),
           "purchase_date":fake.date_time(),
            "purchase_id":fake.pyfloat(),
             "sales":fake.pyfloat()
           }]
    return row
panda = pd.DataFrame(generate_data(2))
fake_data = spark.createDataFrame(panda)
fake_data.write.format("parquet").mode("overwrite").save("/Users/saisundarmasetty/Documents/data_architect_ws/chap3-lab")

In [17]:
# Create a bronze table from the dataset. The data should be streaming but set up to trigger once.
location = "/Users/saisundarmasetty/Documents/data_architect_ws/chap3-lab"
format = "parquet"
schema = spark.read.format(format).load(location).schema
users = spark.readStream.schema(schema).format(format).load(location)

AnalysisException: Unable to infer schema for Parquet at . It must be specified manually.

In [11]:
bronze_schema = users.schema
bronze_location = "/Users/saisundarmasetty/Documents/data_architect_ws/chap3-lab"
bronze_format = "parquet"
checkpoint_location = "/Users/saisundarmasetty/Documents/data_architect_ws/chap3-lab/checkpoint"
output_mode = "append"
bronze_query = users.writeStream.format("parquet").trigger(once=True).option("checkpointLocation", checkpoint_location).option("path", bronze_location).option("mode" ,output_mode).start()
spark.read.format("parquet").load(bronze_location).show()

25/07/01 18:22:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


AnalysisException: Unable to infer schema for Parquet at . It must be specified manually.